In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

Here, we're going to read in the data from csv files.  A lending club investor was kind enough to help me get this data from lending club, and the list below comprises the file names that I chose to give the csv files. We're trying to predict whether or not the loan will be charged off, so we delete the fields that would reveal, after the fact, that the loan has been charged off. We also delete any columns that are null across the entire dataset.

In [2]:
dfs = []
fnames = ['LC1213.csv','LC14.csv','LC15.csv','LC16Q1.csv','LC16Q2.csv','LC16Q3.csv','LC16Q4.csv','LC17Q1.csv','LC17Q2.csv','LC17Q3.csv']
for i in fnames:
    dfs.append(pd.read_csv(i))
print('Done with Loading')
for i in dfs:
    del i['id']
    del i['url']
    del i['recoveries']
    del i['collection_recovery_fee']
    del i['debt_settlement_flag']
    del i['debt_settlement_flag_date']
    del i['settlement_status']
    del i['settlement_date']
    del i['settlement_amount']
    del i['settlement_percentage']
    del i['settlement_term']
    for st in i.columns.values:
        if len(i[np.invert(i[st].isnull())])==0 and len(i[i[st].isnull()])==len(i):
            del i[st]
        else:
            a = i.dropna(subset = [st])
            if(len(a[st].unique())==1):
                del i[st]

/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/IPython/core/int

Done with Loading


Here, we keep features that are common across all of our datasets.

In [3]:
feat = [x for x in dfs[1].columns.values if x in dfs[0].columns.values]
for i in range(2,len(dfs)):
    feat = [x for x in dfs[i].columns.values if x in feat]
for i in dfs:
    for j in i.columns.values:
        if j not in feat:
            del i[j]
df = pd.concat(dfs)

Here, we want to keep the loans that have either been fully paid or charged off. If a loan is still active, it's not helpful to train or test our model, since we don't know whether that loan will be fully paid or charged off.  Let's also take this opportunity to once again delete any features that consist entirely of null values.

In [4]:
df = df[df.loan_status.isin(['Fully Paid', 'Charged Off'])]

In [5]:
for st in df.columns.values:
    if len(df[np.invert(df[st].isnull())])==0 and len(df[df[st].isnull()])==len(df):
        del df[st]
    else:
        a = df.dropna(subset = [st])
        if(len(a[st].unique())==1):
            del df[st]

Here we'll get some features in the format we want and delete all features we couldn't find on Lending Club's investor interface.  The idea is to only use features a casual investor would be able to see.

In [6]:
df['int_rate'] = [float(x.split('%')[0]) for x in df['int_rate']]
dels = ['hardship_reason', 'hardship_status','hardship_amount','hardship_start_date','hardship_end_date','payment_plan_start_date'\
                ,'hardship_dpd', 'hardship_loan_status','orig_projected_additional_accrued_interest'\
                ,'hardship_payoff_balance_amount','hardship_last_payment_amount']
for i in dels:
    df = df[pd.isnull(df[i])]
    del df[i]
df = df[df.loan_amnt == df.funded_amnt]
del df['funded_amnt']
del df['funded_amnt_inv']
del df['grade']
del df['title']
del df['addr_state']
del df['initial_list_status']
del df['total_pymnt_inv']
del df['last_credit_pull_d']
del df['tot_coll_amt']
del df['total_rev_hi_lim']
del df['acc_open_past_24mths']
del df['avg_cur_bal']
del df['bc_open_to_buy']
del df['bc_util']
del df['chargeoff_within_12_mths']
del df['mo_sin_old_il_acct']
del df['mo_sin_old_rev_tl_op']
del df['mo_sin_rcnt_rev_tl_op']
del df['mo_sin_rcnt_tl']
del df['mort_acc']
del df['mths_since_recent_bc']
del df['mths_since_recent_bc_dlq']
del df['mths_since_recent_inq']
del df['num_actv_bc_tl']
del df['num_actv_rev_tl']
del df['mths_since_recent_revol_delinq']
del df['num_accts_ever_120_pd']
del df['num_rev_tl_bal_gt_0']
del df['num_sats']
del df['num_tl_30dpd']
del df['num_tl_120dpd_2m']
del df['num_tl_90g_dpd_24m']
del df['num_bc_tl']
del df['num_il_tl']
del df['num_op_rev_tl']
del df['num_rev_accts']
del df['num_bc_sats']
del df['total_pymnt']
del df['total_rec_prncp']
del df['total_rec_int']
del df['total_rec_late_fee']
del df['last_pymnt_d']
del df['last_pymnt_amnt']
del df['last_fico_range_high']
del df['last_fico_range_low']
del df['tot_cur_bal']
del df['num_tl_op_past_12m']
del df['pct_tl_nvr_dlq']
del df['percent_bc_gt_75']
del df['pub_rec_bankruptcies']
del df['tax_liens']
del df['tot_hi_cred_lim']
del df['total_bal_ex_mort']
del df['total_bc_limit']
del df['total_il_high_credit_limit']
del df['emp_title']
del df['zip_code']

In [8]:
def dtifill(x,y):
    if(pd.isnull(x) and y==0.0):
        return 10000000000000.0
    else:
        return x
df['dti'] = [dtifill(x,y) for x,y in zip(df['dti'],df['annual_inc'])]
def dtifill2(x):
    if pd.isnull(x):
        return 10000000000000.0
    else:
        return x
df['dti'] = [dtifill2(x) for x in df['dti']]
def mfill(x):
    if pd.isnull(x):
        return 100000.0
    else:
        return x
df['mths_since_last_delinq'] = [mfill(x) for x in df['mths_since_last_delinq']]
df['mths_since_last_record'] = [mfill(x) for x in df['mths_since_last_record']]
df['mths_since_last_major_derog'] = [mfill(x) for x in df['mths_since_last_major_derog']]
def utilify(x):
    if pd.isnull(x):
        return x
    else:
        return float(x.split('%')[0])
df['revol_util'] = [utilify(x) for x in df['revol_util']]
def revutilify(x):
    if(pd.isnull(x)):
        return 1000000.0
    else:
        return x
df['revol_util'] = [revutilify(x) for x in df['revol_util']]
def termify(x):
    return int(x.split(' ')[1])
df['term'] = [termify(x) for x in df['term']]
empify = {'10+ years':10.1,'4 years':4.0,'2 years':2.0,'5 years':5.0,'7 years':7.0,'3 years':3.0,\
         '6 years':6.0,'1 year':1.0,'9 years':9.0,'8 years':8.0,'< 1 year':0.5, 'n/a':0.0}
df['emp_length'] = [empify[x] for x in df['emp_length']]
m2i = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
def d2i(x):
    y = x.split('-')
    return m2i[y[0]]+12*(int(y[1])-2012)
df['issue_d'] = [d2i(x) for x in df['issue_d']]
m2i2 = {'Jan':12,'Feb':11,'Mar':10,'Apr':9,'May':8,'Jun':7,'Jul':6,'Aug':5,'Sep':4,'Oct':3,'Nov':2,'Dec':1}
def e2i(x):
    z = x.split('-')
    return m2i2[z[0]] + 12*(2011-int(z[1]))
df['earliest_cr_line'] = [e2i(x) for x in df['earliest_cr_line']]
df['earliest_cr_line'] = [x+y for x,y in zip(df['earliest_cr_line'], df['issue_d'])]
def lstat(x):
    if(x=='Fully Paid'):
        return 0
    elif(x=='Charged Off'):
        return 1
    else:
        return -1
df['loan_status'] = [lstat(x) for x in df['loan_status']]

In [9]:
df.reset_index(inplace = True)
df.dropna(inplace=True)
del df['index']

In [11]:
with open('LCDC3.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

Let's do our monkey test! We'll simply shove our data into 3 models and see what the results are like.

In [12]:
varss = []
for i in df.columns.values:
    if i != 'loan_status':
        varss.append(i)
y = df['loan_status']
X = df[varss]

In [13]:
X = pd.get_dummies(X)

In [14]:
def ret(y_pred, y_test, X_test):
    portfolio = []
    rates = []
    for i in range(len(y_pred)):
        if y_pred[i]==0:
            portfolio.append(y_test[i])
            rates.append(X_test[i])
    if len(portfolio)==0:
        return float(-1)
    return ((float(len(portfolio)-sum(portfolio))/float(len(portfolio)))*(1.+ (np.mean(rates)/100.))) - 1
def myScore(estimator, X, y):
    y_pred = estimator.predict(X)
    return ret(list(y_pred), list(y), list(X['int_rate']))

In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
X_train, X_test, y_train, y_test = train_test_split(X, y)
monkey = BaggingClassifier()
monkey.fit(X_train, y_train)
y_pred = monkey.predict(X_test)
print('Accuracy: ', end = '')
print(metrics.accuracy_score(y_test, y_pred))
print('Precision: ', end = '')
print(metrics.precision_score(y_test, y_pred))
print('Recall: ', end = '')
print(metrics.recall_score(y_test, y_pred))
print('F1: ', end = '')
print(metrics.f1_score(y_test, y_pred))
print('Rate of return: ', end = '')
print(myScore(monkey, X_test, y_test))

Accuracy: 0.781693007187
Precision: 0.443293779216
Recall: 0.145363644939
F1: 0.218934806159
Rate of return: -0.087007910152


In [15]:
from sklearn.linear_model import LogisticRegression
monkey4 = LogisticRegression()
monkey4.fit(X_train, y_train)
y_pred = monkey4.predict(X_test)
print('Accuracy: ', end = '')
print(metrics.accuracy_score(y_test, y_pred))
print('Precision: ', end = '')
print(metrics.precision_score(y_test, y_pred))
print('Recall: ', end = '')
print(metrics.recall_score(y_test, y_pred))
print('F1: ', end = '')
print(metrics.f1_score(y_test, y_pred))
print('Rate of return: ', end = '')
print(myScore(monkey4, X_test, y_test))

Accuracy: 0.790687169916
Precision: 0.0
Recall: 0.0
F1: 0.0
Rate of return: 

/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


-0.10234325529


In [18]:
from sklearn.ensemble import RandomForestClassifier
monkey5 = RandomForestClassifier()
monkey5.fit(X_train, y_train)
y_pred = monkey5.predict(X_test)
print('Accuracy: ', end = '')
print(metrics.accuracy_score(y_test, y_pred))
print('Precision: ', end = '')
print(metrics.precision_score(y_test, y_pred))
print('Recall: ', end = '')
print(metrics.recall_score(y_test, y_pred))
print('F1: ', end = '')
print(metrics.f1_score(y_test, y_pred))
print('Rate of return: ', end = '')
print(myScore(monkey5, X_test, y_test))

Accuracy: 0.785207282691
Precision: 0.461125525633
Recall: 0.121545137251
F1: 0.192381627816
Rate of return: -0.0894826525892
